# SoS Singularity Guide

Before you run any script using singularity, please check if `singularity` is installed. Also, although you can use singularity following this tutorial, it would be really helpful for you to read the [sigularity user's guide](https://www.sylabs.io/docs/) or some online tutorial to understand how singularity works.


Although running singularity images does not need root privilege, building singularity images often requires `sudo` access. So before running any `singularity_build` action with `sudo=True`, please run `sudo -i` to enter sudo, or add your user as a sudo user without password (google "sudo without password" for instructions). The latter is not safe but can be very convenient if you are use singularity a lot.

In [1]:
%revisions

Revision,Author,Date,Message
b4a418a,Bo Peng,2018-08-07,Updata sos docker guide with information on docker toolbox vatlab/sos#996
14318e8,Bo Peng,2018-06-24,Update docker and dryrun related documents
0838fb0,Bo Peng,2018-06-24,Update docker toturial on docker build
8dccbc9,Bo Peng,2018-06-11,Update tutorials on docker
bd54b04,Bo Peng,2018-05-29,Minor update to docker doc
a677605,Bo Peng,2018-05-29,Fix new lines in pexpect output vatlab/sos-notebook#82
dd1d51e,Bo Peng,2018-05-29,Update docker tutorial
e6d96bb,Bo Peng,2018-05-29,Update docker guide
0a5dddc,Bo Peng,2017-10-05,Move sos docs from sos to sos-docs repository


## Running a script using singularity

SoS executes scripts inside a singularity by calling command `singularity exec` with appropriate parameters. For example, if you specify a container with a `shub:` schema, sos will first pull the image, save it as a local image, and use `singularity exec` to run it:

In [1]:
%run
run: container='shub://singularityhub/ubuntu'
  cat /etc/os-release

Pulling singularity image shub://singularityhub/ubuntu


cat /etc/os-release
NAME="Ubuntu"
VERSION="14.04, Trusty Tahr"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 14.04 LTS"
VERSION_ID="14.04"
HOME_URL="http://www.ubuntu.com/"
SUPPORT_URL="http://help.ubuntu.com/"
BUG_REPORT_URL="http://bugs.launchpad.net/ubuntu/"


The actual `singularity exec` command executed by SoS can be shown when you execute the script in dryrun mode (with `-n` option). In this mode, SoS would print the interpolated script (if option `expand=True` is set) and the docker command to execute it:

In [2]:
%rerun -n

singularity exec singularityhub-ubuntu.simg /bin/bash -ev /home/bpeng1/sos/sos-docs/src/tutorials/tmpr76ii92q/singularity_run_53574.sh


cat /etc/os-release



As you can see, the docker command looks similar to

```
singularity exec singularityhub-ubuntu.simg /bin/bash -ev /home/bpeng1/sos/sos-docs/src/tutorials/tmpr76ii92q/singularity_run_53574.sh
```

Basically, SoS pulls the image and runs command `singularity exec` to execte the script with `/bin/bash` inside the image.

You can also use a docker image. However, because `docker://` images are by default executed by docker, you will need to specify the use of singularity using parameter `engine='singularity'`:

In [1]:
%run
run: container='docker://ubuntu', engine='singularity'
  cat /etc/os-release

Pulling image docker://ubuntu


cat /etc/os-release
NAME="Ubuntu"
VERSION="18.04.1 LTS (Bionic Beaver)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 18.04.1 LTS"
VERSION_ID="18.04"
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
VERSION_CODENAME=bionic
UBUNTU_CODENAME=bionic


## Building singularity images

### Action `singularity_build` <a id="singularity_build"></a>

Action `singularity_build` calls command `singularity build` with appropriate command line options. The SoS equivalence of the sample command

```
singularity build lolcow.simg shub://GodloveD/lolcow
```
is the `singularity_build` action with options `src` and `dest`:

In [2]:
singularity_build(src='shub://GodloveD/lolcow', dest='lolcow.simg')

Cache folder set to /home/bpeng1/.singularity/shub
Progress |===================================| 100.0% 
cat: write error: Broken pipe
Building from local image: /home/bpeng1/.singularity/shub/GodloveD-lolcow-master-latest.simg
(BWARNING: it may be missing some functionality.
(BBuilding Singularity image...
Singularity container built: lolcow.simg
Cleaning up...


0

If you need to build the image with root privilege, you can add option `sudo=True` when you are certain that the user is in SUDO (no password) mode:

In [3]:
singularity_build(src='shub://GodloveD/lolcow', dest='lolcow_sudo.simg', sudo=True)

Cache folder set to /root/.singularity/shub
Progress |===================================| 100.0% 
cat: write error: Broken pipe
Building from local image: /root/.singularity/shub/GodloveD-lolcow-master-latest.simg
Building Singularity image...
Singularity container built: lolcow_sudo.simg
Cleaning up...


0

You can also use action `singularity_build` to build an image from a singularity definition file. Using the example from the user's guide:

In [4]:
singularity_build: dest='lolcow.simg', sudo=True
Bootstrap: docker
From: ubuntu:16.04
%post
    apt-get -y update
    apt-get -y install fortune cowsay lolcat
%environment
    export LC_ALL=C
    export PATH=/usr/games:$PATH
%runscript
    fortune | cowsay | lolcat 

Building into existing container: lolcow.simg
Using container recipe deffile: /home/bpeng1/sos/sos-docs/src/tutorials/tmpdv2x3zlv/singularity.def
Sanitizing environment
Adding base Singularity environment to container
User defined %runscript found! Taking priority.
Adding environment to container
Running post scriptlet
+ apt-get -y update
Hit:1 http://archive.ubuntu.com/ubuntu xenial InRelease
Get:2 http://security.ubuntu.com/ubuntu xenial-security InRelease [107 kB]
Get:3 http://archive.ubuntu.com/ubuntu xenial-updates InRelease [109 kB]       
Get:4 http://archive.ubuntu.com/ubuntu xenial-backports InRelease [107 kB]     
Get:5 http://security.ubuntu.com/ubuntu xenial-security/universe Sources [90.3 kB]
Get:6 http://archive.ubuntu.com/ubuntu xenial-updates/universe Sources [277 kB]
Get:7 http://security.ubuntu.com/ubuntu xenial-security/main amd64 Packages [703 kB]
Get:8 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 Packages [1088 kB]
Get:9 http://security.ubuntu.com/ubu

0

Options such as `notest=True` could be add to the action.

## Running script with singularity image (`singularity exec`)

### Binding directories (option `bind`)

## Using singularity image as command (`singularity run`)